In [125]:
import os 
import glob 

import numpy as np
import pandas as pd
import xarray as xr
import netCDF4

from scipy.interpolate import interpn

import matplotlib.pyplot as plt
from   matplotlib.lines import Line2D

In [2]:
#### 
#  Open geogrid file
file     = '/home/sbarc/wrf/wrf401/sbareg/geo_em.d04.nc'
ds       = xr.open_dataset(file)
ds

#   read one 2DVar file to get coordinates
path     = '/home/sbarc/wrf/wrf401/sbareg/'
yr1      = 1988
m        = 0
files    = sorted(glob.glob(path + str(yr1+m) + '/wrfout_d04_2DVar_*.nc'))
f2       = xr.open_dataset(files[m])

#   creat xarray
terrain  = xr.DataArray(ds['HGT_M'][0,:,:].values, dims =("south_north","west_east"), name="terrain",     
                    coords={"XLAT":f2['XLAT'],"XLONG":f2['XLONG']},
                    attrs ={'terrain':'elevation (m)'})
#   creat landmask
landmask = xr.DataArray(ds['LANDMASK'][0,:,:].values, dims =("south_north","west_east"), name="landmask",     
                    coords={"XLAT":f2['XLAT'],"XLONG":f2['XLONG']},
                    attrs ={'landmask':'landmask (m)'})

In [3]:
f2

<xarray.Dataset>
Dimensions:   (Time: 24, bottom_top: 54, south_north: 234, west_east: 258)
Coordinates:
    XLAT      (south_north, west_east) float32 ...
    XLONG     (south_north, west_east) float32 ...
Dimensions without coordinates: Time, bottom_top, south_north, west_east
Data variables:
    HGT       (south_north, west_east) float32 ...
    Times     (Time) |S19 ...
    Q2        (Time, south_north, west_east) float32 ...
    T2        (Time, south_north, west_east) float32 ...
    TH2       (Time, south_north, west_east) float32 ...
    PSFC      (Time, south_north, west_east) float32 ...
    U10       (Time, south_north, west_east) float32 ...
    V10       (Time, south_north, west_east) float32 ...
    FGDP      (Time, south_north, west_east) float32 ...
    DFGDP     (Time, south_north, west_east) float32 ...
    VDFG      (Time, south_north, west_east) float32 ...
    RAINNC    (Time, south_north, west_east) float32 ...
    I_RAINNC  (Time, south_north, west_east) int32 ...
    CLDFRA    (Time, bottom_top, south_north, west_east) float32 ...
    LWUPB     (Time, south_north, west_east) float32 ...
    LWDNB     (Time, south_north, west_east) float32 ...
    SWUPB     (Time, south_north, west_east) float32 ...
    SWDNB     (Time, south_north, west_east) float32 ...
    UST       (Time, south_north, west_east) float32 ...
    HFX       (Time, south_north, west_east) float32 ...
    QFX       (Time, south_north, west_east) float32 ...
    LH        (Time, south_north, west_east) float32 ...
    GRDFLX    (Time, south_north, west_east) float32 ...
    PBLH      (Time, south_north, west_east) float32 ...
Attributes:
    TITLE:                            OUTPUT FROM WRF V4.0.1 MODEL
    START_DATE:                      1987-07-01_00:00:00
    SIMULATION_START_DATE:           1987-07-01_00:00:00
    WEST-EAST_GRID_DIMENSION:        259
    SOUTH-NORTH_GRID_DIMENSION:      235
    BOTTOM-TOP_GRID_DIMENSION:       55
    DX:                              1000.0
    DY:                              1000.0
    AERCU_OPT:                       0
    AERCU_FCT:                       1.0
    IDEAL_CASE:                      0
    DIFF_6TH_SLOPEOPT:               0
    AUTO_LEVELS_OPT:                 2
    DIFF_6TH_THRESH:                 0.1
    DZBOT:                           50.0
    DZSTRETCH_S:                     1.3
    DZSTRETCH_U:                     1.1
    SKEBS_ON:                        0
    SPEC_BDY_FINAL_MU:               1
    USE_Q_DIABATIC:                  0
    GRIDTYPE:                        C
    DIFF_OPT:                        2
    KM_OPT:                          4
    DAMP_OPT:                        3
    DAMPCOEF:                        0.2
    KHDIF:                           0.0
    KVDIF:                           0.0
    MP_PHYSICS:                      6
    RA_LW_PHYSICS:                   4
    RA_SW_PHYSICS:                   4
    SF_SFCLAY_PHYSICS:               5
    SF_SURFACE_PHYSICS:              4
    BL_PBL_PHYSICS:                  5
    CU_PHYSICS:                      0
    SF_LAKE_PHYSICS:                 0
    SURFACE_INPUT_SOURCE:            1
    SST_UPDATE:                      1
    GRID_FDDA:                       0
    GFDDA_INTERVAL_M:                0
    GFDDA_END_H:                     0
    GRID_SFDDA:                      0
    SGFDDA_INTERVAL_M:               0
    SGFDDA_END_H:                    0
    HYPSOMETRIC_OPT:                 2
    USE_THETA_M:                     1
    GWD_OPT:                         0
    SF_URBAN_PHYSICS:                0
    SF_SURFACE_MOSAIC:               0
    SF_OCEAN_PHYSICS:                0
    SHCU_PHYSICS:                    0
    MFSHCONV:                        0
    FEEDBACK:                        1
    SMOOTH_OPTION:                   0
    SWRAD_SCAT:                      1.0
    W_DAMPING:                       0
    DT:                              3.3333333
    RADT:                            10.0
    BLDT:      

In [111]:
xrstack = f2.sel(Time=0)
xlat = 34.001033
xlon = -119.817817
interp=True

# Find closest WRF cell to station
slat = xrstack['XLAT'].values
slon = xrstack['XLONG'].values
dist = np.sqrt((slat - xlat)**2 + (slon - xlon)**2)
ind  = np.unravel_index(np.argmin(dist, axis=None), dist.shape)
np.shape(slat),np.shape(slon),np.shape(dist),ind

# Extract data - interpolate or use nearest neighbor
if interp == True:
    darray = xrstack.interp(south_north=ind[0], west_east=ind[1], method='linear')
    # Drop variables
    darray = darray.drop('south_north')
    darray = darray.drop('west_east')
else:
    darray = xrstack[:,ind[0],ind[1]]

darray  = darray.drop('XLAT')
darray  = darray.drop('XLONG')
df = darray.to_dataframe()

In [ ]:
xlat = 33.995943,
xlon = -119.716437

In [ ]:
tmp = xrstack
lats = tmp['XLAT'].values[:,0]
lons = tmp['XLONG'][0]

tmp = tmp.assign_coords({'south_north':('south_north', lats)})
tmp = tmp.assign_coords({'west_east':('west_east', lons)})
tmp

<xarray.Dataset>
Dimensions:      (bottom_top: 54, south_north: 234, west_east: 258)
Coordinates:
    XLAT         (south_north, west_east) float32 33.51 33.51 ... 35.5 35.5
    XLONG        (south_north, west_east) float32 -121.3 -121.2 ... -118.6
  * south_north  (south_north) float32 33.51 33.52 33.53 ... 35.49 35.49 35.5
  * west_east    (west_east) float32 -121.3 -121.2 -121.2 ... -118.6 -118.6
Dimensions without coordinates: bottom_top
Data variables:
    HGT          (south_north, west_east) float32 0.0 0.0 ... 1.523e+03
    Times        |S19 b'1987-07-01_00:00:00'
    Q2           (south_north, west_east) float32 0.00889 0.00889 ... 0.008336
    T2           (south_north, west_east) float32 289.2 289.2 ... 304.5 304.5
    TH2          (south_north, west_east) float32 288.1 288.2 ... 317.4 318.8
    PSFC         (south_north, west_east) float32 1.015e+05 ... 8.54e+04
    U10          (south_north, west_east) float32 4.428 4.434 ... 4.011 3.995
    V10          (south_north, west_east) float32 -2.56 -2.555 ... 1.824 1.852
    FGDP         (south_north, west_east) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    DFGDP        (south_north, west_east) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    VDFG         (south_north, west_east) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    RAINNC       (south_north, west_east) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    I_RAINNC     (south_north, west_east) int32 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    CLDFRA       (bottom_top, south_north, west_east) float32 ...
    LWUPB        (south_north, west_east) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    LWDNB        (south_north, west_east) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    SWUPB        (south_north, west_east) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    SWDNB        (south_north, west_east) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    UST          (south_north, west_east) float32 0.0001 0.0001 ... 0.0001
    HFX          (south_north, west_east) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    QFX          (south_north, west_east) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    LH           (south_north, west_east) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    GRDFLX       (south_north, west_east) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    PBLH         (south_north, west_east) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes:
    TITLE:                            OUTPUT FROM WRF V4.0.1 MODEL
    START_DATE:                      1987-07-01_00:00:00
    SIMULATION_START_DATE:           1987-07-01_00:00:00
    WEST-EAST_GRID_DIMENSION:        259
    SOUTH-NORTH_GRID_DIMENSION:      235
    BOTTOM-TOP_GRID_DIMENSION:       55
    DX:                              1000.0
    DY:                              1000.0
    AERCU_OPT:                       0
    AERCU_FCT:                       1.0
    IDEAL_CASE:                      0
    DIFF_6TH_SLOPEOPT:               0
    AUTO_LEVELS_OPT:                 2
    DIFF_6TH_THRESH:                 0.1
    DZBOT:                           50.0
    DZSTRETCH_S:                     1.3
    DZSTRETCH_U:                     1.1
    SKEBS_ON:                        0
    SPEC_BDY_FINAL_MU:               1
    USE_Q_DIABATIC:                  0
    GRIDTYPE:                        C
    DIFF_OPT:                        2
    KM_OPT:                          4
    DAMP_OPT:                        3
    DAMPCOEF:                        0.2
    KHDIF:                           0.0
    KVDIF:                           0.0
    MP_PHYSICS:                      6
    RA_LW_PHYSICS:                   4
    RA_SW_PHYSICS:                   4
    SF_SFCLAY_PHYSICS:               5
    SF_SURFACE_PHYSICS:              4
    BL_PBL_PHYSICS:                  5
    CU_PHYSICS:                      0
    SF_LAKE_PHYSICS:                 0
    SURFACE_INPUT_SOURCE:            1
    SST_UPDATE:                      1
    GRID_FDDA:                       0
    GFDDA_INTERVAL_M:                0
    GFDDA_END_H:                     0
    GRID_SFDDA:                      0
    SGFDDA_INT

In [248]:
def interp_value(xrstack, dist, target_lat, target_lon):
    # Assign labels to coords
    ds = xrstack
    lats = ds['XLAT'].values[:,0]
    lons = ds['XLONG'][0]

    ds = ds.assign_coords({'south_north':('south_north', lats)})
    ds = ds.assign_coords({'west_east':('west_east', lons)})

    # Find 4 nearest points to target
    flat_indices = np.argsort(dist.ravel())[:3]
    indices = np.unravel_index(flat_indices, dist.shape)

    # Get bounding box to crop dataset
    near_lats = ds['XLAT'].values[indices[0], indices[1]]
    near_lons = ds['XLONG'].values[indices[0], indices[1]]

    max_lat = np.max(near_lats)
    min_lat = np.min(near_lats)
    max_lon = np.max(near_lons)
    min_lon = np.min(near_lons)

    # Crop dataset to locality of target
    ds_local = ds.sel(south_north=slice(min_lat,max_lat), west_east=slice(min_lon,max_lon))
    
    # Interpolate using local dataset
    interp_value = ds_local.interp(south_north=target_lat, west_east=target_lon, method='linear')

    return ds_local

xrstack = f2.sel(Time=0)
xlat = 33.995943
xlon = -119.716437
interp=True

# Find closest WRF cell to station
slat = xrstack['XLAT'].values
slon = xrstack['XLONG'].values
dist = np.sqrt((slat - xlat)**2 + (slon - xlon)**2)
ind  = np.unravel_index(np.argmin(dist, axis=None), dist.shape)
np.shape(slat),np.shape(slon),np.shape(dist),ind
interp_value(xrstack, dist, xlat, xlon)

<xarray.Dataset>
Dimensions:      (bottom_top: 54, south_north: 2, west_east: 2)
Coordinates:
    XLAT         (south_north, west_east) float32 33.99 33.99 34.0 34.0
    XLONG        (south_north, west_east) float32 -119.7 -119.7 -119.7 -119.7
  * south_north  (south_north) float32 33.99 34.0
  * west_east    (west_east) float32 -119.7 -119.7
Dimensions without coordinates: bottom_top
Data variables:
    HGT          (south_north, west_east) float32 156.3 163.1 181.2 157.0
    Times        |S19 b'1987-07-01_00:00:00'
    Q2           (south_north, west_east) float32 ...
    T2           (south_north, west_east) float32 290.5 290.5 290.5 290.5
    TH2          (south_north, west_east) float32 ...
    PSFC         (south_north, west_east) float32 ...
    U10          (south_north, west_east) float32 ...
    V10          (south_north, west_east) float32 ...
    FGDP         (south_north, west_east) float32 ...
    DFGDP        (south_north, west_east) float32 ...
    VDFG         (south_north, west_east) float32 ...
    RAINNC       (south_north, west_east) float32 ...
    I_RAINNC     (south_north, west_east) int32 ...
    CLDFRA       (bottom_top, south_north, west_east) float32 ...
    LWUPB        (south_north, west_east) float32 ...
    LWDNB        (south_north, west_east) float32 ...
    SWUPB        (south_north, west_east) float32 ...
    SWDNB        (south_north, west_east) float32 ...
    UST          (south_north, west_east) float32 ...
    HFX          (south_north, west_east) float32 ...
    QFX          (south_north, west_east) float32 ...
    LH           (south_north, west_east) float32 ...
    GRDFLX       (south_north, west_east) float32 ...
    PBLH         (south_north, west_east) float32 ...
Attributes:
    TITLE:                            OUTPUT FROM WRF V4.0.1 MODEL
    START_DATE:                      1987-07-01_00:00:00
    SIMULATION_START_DATE:           1987-07-01_00:00:00
    WEST-EAST_GRID_DIMENSION:        259
    SOUTH-NORTH_GRID_DIMENSION:      235
    BOTTOM-TOP_GRID_DIMENSION:       55
    DX:                              1000.0
    DY:                              1000.0
    AERCU_OPT:                       0
    AERCU_FCT:                       1.0
    IDEAL_CASE:                      0
    DIFF_6TH_SLOPEOPT:               0
    AUTO_LEVELS_OPT:                 2
    DIFF_6TH_THRESH:                 0.1
    DZBOT:                           50.0
    DZSTRETCH_S:                     1.3
    DZSTRETCH_U:                     1.1
    SKEBS_ON:                        0
    SPEC_BDY_FINAL_MU:               1
    USE_Q_DIABATIC:                  0
    GRIDTYPE:                        C
    DIFF_OPT:                        2
    KM_OPT:                          4
    DAMP_OPT:                        3
    DAMPCOEF:                        0.2
    KHDIF:                           0.0
    KVDIF:                           0.0
    MP_PHYSICS:                      6
    RA_LW_PHYSICS:                   4
    RA_SW_PHYSICS:                   4
    SF_SFCLAY_PHYSICS:               5
    SF_SURFACE_PHYSICS:              4
    BL_PBL_PHYSICS:                  5
    CU_PHYSICS:                      0
    SF_LAKE_PHYSICS:                 0
    SURFACE_INPUT_SOURCE:            1
    SST_UPDATE:                      1
    GRID_FDDA:                       0
    GFDDA_INTERVAL_M:                0
    GFDDA_END_H:                     0
    GRID_SFDDA:                      0
    SGFDDA_INTERVAL_M:               0
    SGFDDA_END_H:                    0
    HYPSOMETRIC_OPT:                 2
    USE_THETA_M:                     1
    GWD_OPT:                         0
    SF_URBAN_PHYSICS:                0
    SF_SURFACE_MOSAIC:               0
    SF_OCEAN_PHYSICS:                0
    SHCU_PHYSICS:                    0
    MFSHCONV:                        0
    FEEDBACK:                        1
    SMOOTH_OPTION:                   0
    SWRAD_SCAT:                      1.0
    W_DAMPING:                       0
    

In [ ]:
ind  = np.unravel_index(np.argmin(dist, axis=None), dist.shape)

33.99224
-119.71664
